# Setup LLM

In [1]:
!pip install transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cpu" # the device to load the model onto

# model_name = "google/gemma-2b-it"
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model_name = "Qwen/Qwen2-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, token = '<your_token_here>')
tokenizer = AutoTokenizer.from_pretrained(model_name, token = '<your_token_here>')
model.to(device);

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Risk Profile + Prompt Creation

In [4]:
risk_profiles = [
    "1. LOW-RISK: You are a conservative investor who is not willing to take on much risk. You are looking for investments that are safe and secure, even if they offer lower returns.",
    "2. MEDIUM-RISK: You are a moderate investor who is willing to take on some risk in exchange for the potential of higher returns. You are looking for a balanced mix of investments that offer a good combination of safety and growth.",
    "3. HIGH-RISK: You are an aggressive investor who is willing to take on a lot of risk in exchange for the potential of higher returns. You are looking for investments that offer the highest possible returns, even if they are more volatile and risky."
]
risk_profiles = '\n'.join(risk_profiles)

# user_first_response = f"I am weathly and looking to invest my money in a conservative portfolio. I am looking for investments that are safe and secure, even if they offer lower returns."
user_first_response = f"I am poor and looking to invest my money in a high growth portfolio. I am looking for investments that are going to earn a lot of money in a short amount of time."
# user_first_response = f"I am not sure how I want to invest my money. The stock market makes me nervous, but I want to make sure my money is working for me."
# user_first_response = f"Purple monkey dishwasher."
# user_risk = input("Hello! What kind of portfolio are you looking to build?")
instruction = f"You are a FINRA registered investment advisor. Your job is to assign me to one of three risk categories: 1. LOW-RISK, 2. MEDIUM-RISK, 3.HIGH-RISK. Given the descriptions of the three risk categories below, assign me to the risk category that best aligns with my description:\n\n### RISK CATEGORIES:\n{risk_profiles}\n\nOnly return the name of the risk category (e.g., 'LOW-RISK')."


messages=[{'role': 'system', 'content': instruction},
            {'role': 'user', 'content': user_first_response},
            ]

In [5]:
print(messages)

[{'role': 'system', 'content': "You are a FINRA registered investment advisor. Your job is to assign me to one of three risk categories: 1. LOW-RISK, 2. MEDIUM-RISK, 3.HIGH-RISK. Given the descriptions of the three risk categories below, assign me to the risk category that best aligns with my description:\n\n### RISK CATEGORIES:\n1. LOW-RISK: You are a conservative investor who is not willing to take on much risk. You are looking for investments that are safe and secure, even if they offer lower returns.\n2. MEDIUM-RISK: You are a moderate investor who is willing to take on some risk in exchange for the potential of higher returns. You are looking for a balanced mix of investments that offer a good combination of safety and growth.\n3. HIGH-RISK: You are an aggressive investor who is willing to take on a lot of risk in exchange for the potential of higher returns. You are looking for investments that offer the highest possible returns, even if they are more volatile and risky.\n\nOnly 

In [6]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=50, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded[0].split(messages[-1]['content'])[-1].strip())

answer
HIGH-RISK


In [7]:
print(user_first_response)
print()
for risk in ['LOW-RISK', 'MEDIUM-RISK', 'HIGH-RISK']:
    if risk.lower() in decoded[0].split(messages[-1]['content'])[-1].strip().lower():
        risk_category = risk
        break
    else:
        risk_category = "UNKNOWN"

print(f'Risk Category: {risk_category}')

I am poor and looking to invest my money in a high growth portfolio. I am looking for investments that are going to earn a lot of money in a short amount of time.

Risk Category: HIGH-RISK


## Portfolio Creation

In [8]:
import pandas as pd
import numpy as np
import random
import itertools

In [11]:
filtered_stocks = pd.read_csv('/snp_portfolio_risk_rsi+macd_filtered.csv')
filtered_stocks.head()

,Stock,Risk_Level,Industry,Beta,Volatility,Debt_to_Equity,PE_Ratio,Earnings_Growth,MACD,Signal,...,2024-04-17 00:00:00,2024-04-18 00:00:00,2024-04-19 00:00:00,2024-04-22 00:00:00,2024-04-23 00:00:00,2024-04-24 00:00:00,2024-04-25 00:00:00,2024-04-26 00:00:00,2024-04-29 00:00:00,2024-04-30 00:00:00
0,ABT,Low,Medical Devices,0.751,0.254548,37.664,33.514020,-0.071,-1.618899,-1.661040,...,105.900002,105.269997,107.279999,107.070000,107.589996,106.889999,106.860001,107.529999,107.269997,105.970001
1,AEE,Low,Utilities - Regulated Electric,0.443,0.216294,148.496,16.300459,-0.011,0.582307,0.485202,...,71.930000,72.510002,73.879997,74.379997,74.019997,74.709999,74.760002,73.660004,74.349998,73.870003
2,AEP,Low,Utilities - Regulated Electric,0.500,0.208181,170.989,16.443203,1.527,0.820753,0.492052,...,80.450737,81.757919,83.381981,84.075188,84.728767,85.530907,86.016144,84.431686,85.827988,85.194206
3,AJG,Low,Insurance Brokers,0.694,0.252346,75.987,51.530487,0.251,-2.659436,-2.695624,...,230.520004,233.990005,236.570007,237.520004,237.869995,236.809998,236.949997,234.110001,233.550003,234.690002
4,ATO,Low,Utilities - Regulated Gas,0.656,0.230794,64.859,17.312216,0.208,0.677511,0.414041,...,114.559998,114.570000,117.199997,118.080002,117.669998,118.660004,118.169998,116.930000,118.639999,117.900002


In [12]:
filtered_stocks['Risk_Level'].value_counts()

Risk_Level
Medium    137
Low        50
High       39
Name: count, dtype: int64

In [13]:
# Ensure required columns are numeric
filtered_stocks['Beta'] = pd.to_numeric(filtered_stocks['Beta'], errors='coerce')
filtered_stocks['Volatility'] = pd.to_numeric(filtered_stocks['Volatility'], errors='coerce')
filtered_stocks['Debt_to_Equity'] = pd.to_numeric(filtered_stocks['Debt_to_Equity'], errors='coerce')
filtered_stocks['PE_Ratio'] = pd.to_numeric(filtered_stocks['PE_Ratio'], errors='coerce')
filtered_stocks['Earnings_Growth'] = pd.to_numeric(filtered_stocks['Earnings_Growth'], errors='coerce')
filtered_stocks['EPS'] = pd.to_numeric(filtered_stocks['EPS'], errors='coerce')
filtered_stocks['Market_Cap (Millions)'] = pd.to_numeric(filtered_stocks['Market_Cap (Millions)'], errors='coerce')
filtered_stocks['Dividend_Yield'] = pd.to_numeric(filtered_stocks['Dividend_Yield'], errors='coerce')
filtered_stocks['MACD'] = pd.to_numeric(filtered_stocks['MACD'], errors='coerce')
filtered_stocks['Signal'] = pd.to_numeric(filtered_stocks['Signal'], errors='coerce')
filtered_stocks['RSI'] = pd.to_numeric(filtered_stocks['RSI'], errors='coerce')

In [14]:
filtered_stocks.shape

(226, 2531)

In [15]:
# Read sentiment data
sentiment_filtered_stocks = pd.read_csv('/news_sentiment_filtered_stocks.csv')
print(sentiment_filtered_stocks.head())

sentiment_filtered_stocks.shape

  Stock Risk_Level                 LastUpdated                   Industry  \
0    ON       High  2024-06-09 09:32:19.507737                   Airlines   
1   AAL       High  2024-06-09 09:32:19.507737       Consumer Electronics   
2  ABNB       High  2024-06-09 09:32:19.507737            Travel Services   
3  PARA       High  2024-06-09 09:32:19.507737  Software - Infrastructure   
4   VTR       High  2024-06-09 09:32:19.507737             Semiconductors   

    Beta  Volatility  Debt_to_Equity    PE_Ratio  Earnings_Growth       MACD  \
0  1.580    0.658142      172.635933   16.197184         0.468858  -0.129848   
1  1.264    0.443116      140.968000   30.572980        -0.022000  -0.861375   
2  1.246    0.508553       29.002000   19.659517         1.256000   0.133508   
3  1.281    0.485039       26.429000   44.453674        -0.503000 -12.908131   
4  1.660    0.592271        5.335000  243.289860         0.468858  -6.084851   

      Signal        RSI    EPS  Market_Cap (Millions)  D

(226, 15)

In [16]:
# Function to calculate daily returns
def calculate_daily_returns(df):
    numeric_columns = df.apply(pd.to_numeric, errors='coerce').select_dtypes(include=[np.number])
    print(f"Numeric columns for returns calculation: {numeric_columns.columns.tolist()}")
    daily_returns = numeric_columns.pct_change().dropna()
    print(f"Daily returns calculated, shape: {daily_returns.shape}")
    return daily_returns


In [17]:
# Function to calculate Value at Risk (VaR)
def calculate_var(returns, confidence_level=0.95):
    if len(returns) == 0:
        return np.nan
    sorted_returns = np.sort(returns)
    index = int((1 - confidence_level) * len(sorted_returns))
    var = sorted_returns[index]
    return var

In [18]:
def filter_stocks(df, risk_level, rsi_max=30, macd_min=0):
    """
    Filters stocks based on risk level, RSI, and MACD values.

    Args:
    df (pd.DataFrame): DataFrame containing stock data.
    risk_level (str): Desired risk level (e.g., 'Low Risk', 'Medium Risk', 'High Risk').
    rsi_max (float): Maximum RSI value for considering a stock.
    macd_min (float): Minimum MACD value for considering a stock.

    Returns:
    pd.DataFrame: Filtered DataFrame with stocks that meet the criteria.
    """
    filtered_df = df[df['Risk_Level'] == risk_level]
    # filtered_df = filtered_df[((filtered_df['RSI'] < rsi_max) |
    #                            (filtered_df['RSI'].between(30, 70))) &
    #                           (filtered_df['MACD'] > macd_min) &
    #                           (filtered_df['MACD'] > filtered_df['Signal'])]
    # filtered_df = filtered_df[(filtered_df['RSI'] < 70) &
    #                           (filtered_df['MACD'] > macd_min)]
    filtered_df = filtered_df[(filtered_df['RSI'] < 70) &
                              (filtered_df['MACD'] > filtered_df['Signal'])]
    return filtered_df

## Get Recommended Portfolio for given risk profile

In [22]:
# Function to get the best portfolio based on VaR
def get_best_portfolio(df, risk_level, num_stocks, sample_size=1000):
    # Filter stocks for particular risk level
    print(f"Filtering stocks for risk level: {risk_level}")
    filtered_df = filter_stocks(df, risk_level)

    if filtered_df.empty:
        print("No stocks found after filtering.")
        return None, np.inf

    # COMMENTED OUT: Uncomment once news sentiment code is working
    # Now filter stocks based on news sentiment
    # stock_symbols = filtered_df['Stock'].values
    # filtered_stocks = filter_stocks_based_on_sentiment(stock_symbols)
    # filtered_df = filtered_df[filtered_df['Stock'].isin(filtered_stocks)]
    # Reading already sentiment filtered stocks
    #TODO: Use the dynamic function that checks if the lastUpdated date is older than 3 days and then makes call to the sentiment function
    sentiment_filtered_stocks = pd.read_csv('/news_sentiment_filtered_stocks.csv')
    # Only keep stocks from the filtered_df in the sentiment_filtered_stocks
    filtered_df = filtered_df[filtered_df['Stock'].isin(sentiment_filtered_stocks['Stock'])]
    print(f"Filtered stocks based on news sentiment: {filtered_df.shape}")

    # Calculate daily returns
    stock_list = filtered_df['Stock'].values
    daily_prices = df.loc[df['Stock'].isin(stock_list)].set_index('Stock').iloc[:, 13:]

    if daily_prices.empty:
        print("No price data available for the filtered stocks.")
        return None, np.inf

    try:
        daily_returns = calculate_daily_returns(daily_prices.T)
        daily_returns = daily_returns.T.reset_index()
        print(f"Daily returns calculated, shape: {daily_returns.shape}")
    except Exception as e:
        print(f"Error calculating daily returns: {e}")
        return None, np.inf

    best_var = np.inf
    best_portfolio = None

    # Randomly sample combinations
    if len(stock_list) < num_stocks:
        print("Not enough stocks to form a portfolio.")
        return None, np.inf

    combinations_checked = 0
    for combination in itertools.combinations(stock_list, num_stocks):
        combinations_checked += 1
        if combinations_checked > sample_size:
            break
        selected_stocks = list(combination)
        try:
            if set(selected_stocks).issubset(daily_returns['Stock']):
                portfolio_returns = daily_returns.set_index('Stock').loc[selected_stocks].mean()
                var = calculate_var(portfolio_returns.values)
                if var < best_var:
                    best_var = var
                    best_portfolio = selected_stocks
            else:
                print(f"Skipping combination {selected_stocks} as they are not in daily returns columns.")
        except Exception as e:
            print(f"Error with combination {selected_stocks}: {e}")

    print(f"Total combinations checked: {combinations_checked}")
    return best_portfolio, best_var


## Call the function to get the recommended portfolio for inferred risk profile

In [23]:
if risk_category == 'HIGH-RISK':
  portfolio_high = get_best_portfolio(filtered_stocks, risk_level='High', num_stocks=10)
  print("Our final recommendation of your stock portfolio is:", portfolio_high)
elif risk_category == 'MEDIUM-RISK':
  portfolio_medium = get_best_portfolio(filtered_stocks, risk_level='Medium', num_stocks=10)
  print("Our final recommendation of your stock portfolio is:", portfolio_medium)
elif risk_category == 'LOW-RISK':
  portfolio_low = get_best_portfolio(filtered_stocks, risk_level='Low', num_stocks=10)
  print("Our final recommendation of your stock portfolio is:", portfolio_low)

Filtering stocks for risk level: High
Filtered stocks based on news sentiment: (39, 2531)
Numeric columns for returns calculation: ['AAL', 'AAPL', 'ABNB', 'ADBE', 'AMD', 'BBWI', 'BLDR', 'CCL', 'COF', 'DAL', 'DAY', 'DFS', 'ETSY', 'EXPE', 'HWM', 'ILMN', 'KLAC', 'LULU', 'MCHP', 'MHK', 'MPWR', 'MRNA', 'MS', 'NCLH', 'NVDA', 'ON', 'PARA', 'PHM', 'QRVO', 'RCL', 'SWKS', 'TECH', 'TPR', 'TRMB', 'TSLA', 'UAL', 'ULTA', 'URI', 'VTR']
Daily returns calculated, shape: (850, 39)
Daily returns calculated, shape: (39, 851)
Total combinations checked: 1001
Our final recommendation of your stock portfolio is: (['AAL', 'AAPL', 'ABNB', 'ADBE', 'AMD', 'BBWI', 'BLDR', 'CCL', 'NCLH', 'TSLA'], -0.03545860163516208)
